<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/getHAFSASlices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program subsets and saves portions of output from the MOM6 model along the track of a given TC.

#Set up environment

In [3]:
!pip install cfgrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 52.3 MB/s eta 0:00:00


In [34]:
!sudo apt install aria2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libc-ares2 amd64 1.18.1-1ubuntu0.22.04.3 [45.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libaria2-0 amd64 1.36.0-1 [1,086 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aria2 amd64 1.36.0-1 [381 kB]
Fetched 1,513 kB in 1s (1,431 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to front

In [35]:
import xarray as xr
import pandas as pd
from datetime import datetime as dt
import cfgrib
import subprocess as sp

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#User parameters

In [42]:
subfolder = "RI"
saveLocAtm = "drive/MyDrive/savedData/hafsaOutput/" + subfolder
saveLocOce = "drive/MyDrive/savedData/mom6Output/" + subfolder
bucket = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/"

name = "Milton"
tcNum = "14"
trackType = ""

start = dt(year=2024, month=10, day=6, hour=0)
end = dt(year=2024, month=10, day=6, hour=0)

fHourStep = 6 #Normally 3 for HAFS-A
forecastLength = 48 #Normally 126 for HAFS-A. Changeable for testing.
runStep = 6 #Normally 6 for HAFS-A

downloadAtm = True
downloadOce = False
replaceExisting = True

In [43]:
atmTop = 700 # in hPa
oceBottom = 530 # In m below surface. This was chosen to include a layer in the files that's around 529 m

#Determine Area and Temporal Extent

Get best track data and find bounds of TC track

In [44]:
bt = pd.read_csv(f"/content/drive/MyDrive/savedData/{name}/hurdat2_" + name + trackType + ".csv")
latBounds = [min(bt.lat), max(bt.lat)]
lonBounds = [min(bt.lon), max(bt.lon)]

In [45]:
dateFormat = "%Y-%m-%d %H:%M:%S"
runFormat = "%Y%m%d%H"

Find times needed

In [46]:
fcastTimes = {} #Key: initiation, item: valid time list

initTime = start
while initTime <= end:
  validTime = initTime
  fcastTimes[initTime] = []
  fhour = 0

  while fhour <= forecastLength:
    fcastTimes[initTime].append(validTime)
    validTime += pd.Timedelta(hours=fHourStep)
    fhour += fHourStep

  initTime += pd.Timedelta(hours=runStep)

In [47]:
t = [fcastTimes[key] for key in fcastTimes.keys()]
flat_list = [item for sublist in t for item in sublist]
totalAtmData = 200 * len(flat_list)
totalOceData = 30 * len(flat_list)
print(f"Approximate storage needed: {(totalAtmData+totalOceData)/1000} GB")

Approximate storage needed: 2.07 GB


In [48]:
fcastTimes

{datetime.datetime(2024, 10, 6, 0, 0): [datetime.datetime(2024, 10, 6, 0, 0),
  datetime.datetime(2024, 10, 6, 6, 0),
  datetime.datetime(2024, 10, 6, 12, 0),
  datetime.datetime(2024, 10, 6, 18, 0),
  datetime.datetime(2024, 10, 7, 0, 0),
  datetime.datetime(2024, 10, 7, 6, 0),
  datetime.datetime(2024, 10, 7, 12, 0),
  datetime.datetime(2024, 10, 7, 18, 0),
  datetime.datetime(2024, 10, 8, 0, 0)]}

#Retrieve Data

Determine what files already exist

In [49]:
existingFilesLs = !ls {saveLocAtm}
existingFiles = [f for files in existingFilesLs for f in files.split("  ")]
existingFileNames = [f.replace("nc", "grb2").replace("hafsa", "atm").replace("mom6", "oce") for f in existingFiles]
existingFileNames.sort()

Download HAFS

In [50]:
for init, validList in fcastTimes.items():

  if not downloadAtm: break

  initDate, initHour = init.strftime("%Y%m%d_%H").split("_")

  for valid in validList:

    fhour = str(int((valid-init).total_seconds() / 3600))
    while len(fhour) < 3: fhour = "0" + fhour

    atmURL = bucket + initDate + "/" + initHour + "/" + tcNum + "l." + initDate + initHour + ".hfsa.storm.atm.f" + fhour + ".grb2"
    atmFile = "atm_" + initDate + initHour + "_f" + fhour + ".grb2"
    if atmFile in existingFileNames and not replaceExisting:
      print(atmFile + " already exists.\n")
      continue #Skip files we already have

    !aria2c -x 16 -s 16 --allow-overwrite=true -o {atmFile} {atmURL}
    isoData = xr.open_dataset(atmFile, engine="cfgrib", decode_timedelta=True, filter_by_keys={'typeOfLevel': 'isobaricInhPa', 'stepType': 'instant'})
    sfcData = xr.open_dataset(atmFile, engine="cfgrib", decode_timedelta=True, filter_by_keys={'typeOfLevel': 'surface', 'stepType': 'instant'})
    atmData = xr.merge([isoData, sfcData], compat="override") # Takes conflicting variables from the isobaric data

    pressureSlice = slice(max(atmData.isobaricInhPa.data), atmTop)
    #Longitude in atm files are in degrees east, but are -180 - 180 in oce files. point has them from -180 - 180
    atmSlice = atmData.sel(latitude=slice(latBounds[0], latBounds[1]), longitude=slice(360+lonBounds[0], 360+lonBounds[1]), isobaricInhPa=pressureSlice)

    atmSlice.to_netcdf(saveLocAtm + "/hafsa_" + initDate + initHour + "_f" + fhour + ".nc")

    !rm {atmFile}


07/09 15:55:39 [NOTICE] Downloading 1 item(s)

07/09 15:55:41 [NOTICE] Download complete: /content/atm_2024100600_f000.grb2

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
f663e2|OK  |   160MiB/s|/content/atm_2024100600_f000.grb2

Status Legend:
(OK):download completed.


ERROR:cfgrib.dataset:skipping variable: paramId==3017 shortName='dpt'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,
        775.,  750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,
        550.,  525.,  500.,  475.,  450.,  425.,  400.,  375.,  350.,
        325.,  300.,  275.,  250.,  225.,  200.,  175.,  150.,  125.,
        100.,   70.,   50.,   30.,   20.,   10.,    7.,    5.,    2.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([850., 700., 500., 300., 200., 100.]))



07/09 15:56:01 [NOTICE] Downloading 1 item(s)

07/09 15:56:03 [NOTICE] Download complete: /content/atm_2024100600_f006.grb2

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
a4a616|OK  |   150MiB/s|/content/atm_2024100600_f006.grb2

Status Legend:
(OK):download completed.


ERROR:cfgrib.dataset:skipping variable: paramId==3017 shortName='dpt'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,
        775.,  750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,
        550.,  525.,  500.,  475.,  450.,  425.,  400.,  375.,  350.,
        325.,  300.,  275.,  250.,  225.,  200.,  175.,  150.,  125.,
        100.,   70.,   50.,   30.,   20.,   10.,    7.,    5.,    2.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([850., 700., 500., 300., 200., 100.]))



07/09 15:56:22 [NOTICE] Downloading 1 item(s)

07/09 15:56:25 [NOTICE] Download complete: /content/atm_2024100600_f012.grb2

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
b1d592|OK  |   167MiB/s|/content/atm_2024100600_f012.grb2

Status Legend:
(OK):download completed.


...

ERROR:cfgrib.dataset:skipping variable: paramId==3017 shortName='dpt'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,
        775.,  750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,
        550.,  525.,  500.,  475.,  450.,  425.,  400.,  375.,  350.,
        325.,  300.,  275.,  250.,  225.,  200.,  175.,  150.,  125.,
        100.,   70.,   50.,   30.,   20.,   10.,    7.,    5.,    2.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([850., 700., 500., 300., 200., 100.]))



07/09 15:58:04 [NOTICE] Downloading 1 item(s)

07/09 15:58:07 [NOTICE] Download complete: /content/atm_2024100600_f042.grb2

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
f83f90|OK  |   182MiB/s|/content/atm_2024100600_f042.grb2

Status Legend:
(OK):download completed.


ERROR:cfgrib.dataset:skipping variable: paramId==3017 shortName='dpt'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,
        775.,  750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,
        550.,  525.,  500.,  475.,  450.,  425.,  400.,  375.,  350.,
        325.,  300.,  275.,  250.,  225.,  200.,  175.,  150.,  125.,
        100.,   70.,   50.,   30.,   20.,   10.,    7.,    5.,    2.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([850., 700., 500., 300., 200., 100.]))



07/09 15:58:27 [NOTICE] Downloading 1 item(s)

07/09 15:58:30 [NOTICE] Download complete: /content/atm_2024100600_f048.grb2

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
49c52e|OK  |   196MiB/s|/content/atm_2024100600_f048.grb2

Status Legend:
(OK):download completed.


ERROR:cfgrib.dataset:skipping variable: paramId==3017 shortName='dpt'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,
        775.,  750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,
        550.,  525.,  500.,  475.,  450.,  425.,  400.,  375.,  350.,
        325.,  300.,  275.,  250.,  225.,  200.,  175.,  150.,  125.,
        100.,   70.,   50.,   30.,   20.,   10.,    7.,    5.,    2.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([850., 700., 500., 300., 200., 100.]))


In [51]:
atmData

<xarray.Dataset> Size: 2GB
Dimensions:        (isobaricInhPa: 45, latitude: 801, longitude: 1001)
Coordinates:
    time           datetime64[ns] 8B ...
    step           timedelta64[ns] 8B ...
  * isobaricInhPa  (isobaricInhPa) float64 360B 1e+03 975.0 950.0 ... 5.0 2.0
  * latitude       (latitude) float64 6kB 14.54 14.56 14.58 ... 30.5 30.52 30.54
  * longitude      (longitude) float64 8kB 259.0 259.0 259.0 ... 279.0 279.0
    valid_time     datetime64[ns] 8B ...
    surface        float64 8B ...
Data variables: (12/36)
    gh             (isobaricInhPa, latitude, longitude) float32 144MB ...
    t              (isobaricInhPa, latitude, longitude) float32 144MB ...
    r              (isobaricInhPa, latitude, longitude) float32 144MB ...
    q              (isobaricInhPa, latitude, longitude) float32 144MB ...
    w              (isobaricInhPa, latitude, longitude) float32 144MB ...
    wz             (isobaricInhPa, latitude, longitude) float32 144MB ...
    ...             ...
    sulwrf         (latitude, longitude) float32 3MB ...
    unknown        (latitude, longitude) float32 3MB ...
    nlat           (latitude, longitude) float32 3MB ...
    elon           (latitude, longitude) float32 3MB ...
    lsm            (latitude, longitude) float32 3MB ...
    sst            (latitude, longitude) float32 3MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2025-07-09T15:58 GRIB to CDM+CF via cfgrib-0.9.1...

Download MOM6

In [52]:
existingFilesLs = !ls {saveLocOce}
existingFiles = [f for files in existingFilesLs for f in files.split("  ")]
existingFileNames = [f.replace("nc", "grb2").replace("hafsa", "atm").replace("mom6", "oce") for f in existingFiles]
existingFileNames.sort()

In [53]:
missingFiles = ['oce_2024100800_f000.nc', "oce_2024100800_f126.nc"]

In [54]:
for init, validList in fcastTimes.items():

  if not downloadOce: break

  initDate, initHour = init.strftime("%Y%m%d_%H").split("_")
  oceRun = False

  for valid in validList:

    fhour = str(int((valid-init).total_seconds() / 3600))
    while len(fhour) < 3: fhour = "0" + fhour

    oceURL = bucket + initDate + "/" + initHour + "/" + tcNum + "l." + initDate + initHour + ".hfsa.mom6.f" + fhour + ".nc"
    oceFile = "oce_" + initDate + initHour + "_f" + fhour + ".nc"

    if atmFile in existingFileNames and not replaceExisting:
      print(atmFile + " already exists.\n")
      continue #Skip files we already have

    if oceFile in missingFiles: continue #This file is missing
    !aria2c -x 16 -s 16 --allow-overwrite=true -o {oceFile} {oceURL}
    oceData = xr.open_dataset(oceFile, decode_times=False)

    depthSlice = slice(min(oceData.z_l.data), oceBottom)
    oceSlice = oceData.sel(z_l=depthSlice, z_i=depthSlice,\
                           xh=slice(lonBounds[0],lonBounds[1]), yh=slice(latBounds[0],latBounds[1]),\
                           xq=slice(lonBounds[0],lonBounds[1]), yq=slice(latBounds[0],latBounds[1]))

    oceSlice.to_netcdf(saveLocOce + "/mom6_" + initDate + initHour + "_f" + fhour + ".nc")

    !rm {oceFile}